In [11]:
import librosa
import numpy as np
import os
import sys
import h5py
import multiprocessing
import functools
import h5py
import threading
from pathlib import Path
from numpy import pi, convolve
from scipy.signal.filter_design import bilinear
from scipy.signal import lfilter
# prep piptrack
fpath=r"C:\Users\gaoyu\Desktop\music try\data_resonance\all_types_F_chain.wav"
y,sr=librosa.load(fpath,sr=conf.sr,res_type="kaiser_fast")
# configure for librosa
class Config(object):
    n_fft = 1024
    hop_length=512
    sr = 22050
    duration = 120
    feature = "mel"

conf = Config()

def extract_max(pitches, magnitudes, shape):
    new_pitches = []
    new_magnitudes = []
    for i in range(0, shape[1]):
        new_pitches.append(np.max(pitches[:,i]))
        new_magnitudes.append(np.max(magnitudes[:,i]))
    return (new_pitches,new_magnitudes)


def smooth(x, window_len=11, window='hanning'):
    if window_len<3:
        return x
    if not window in ['flat', 'hanning', 'hamming', 'bartlett', 'blackman']:
        raise ValueError("Window is on of 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'")
    s=np.r_[2*x[0]-x[window_len-1::-1],x,2*x[-1]-x[-1:-window_len:-1]]
    if window == 'flat': #moving average
        w=np.ones(window_len,'d')
    else:
        w=eval('np.'+window+'(window_len)')
    y=np.convolve(w/w.sum(),s,mode='same')
    return y[window_len:-window_len+1]

def pitch_detect(y, sr):
    pitches, magnitudes = librosa.core.piptrack(y=y, sr=sr)
    shape = pitches.shape
    nb_samples = shape[0]
    nb_windows = shape[1]
    pitches, magnitudes = extract_max(pitches, magnitudes, shape)
    pitches = smooth(pitches, window_len=5)
    return pitches

pip=pitch_detect(y,sr)

In [17]:
'''
def pitch(freq):
    h = round(12*log2(freq/C0))
    octave = h // 12
    n = h % 12
    return name[n] + str(octave)
'''
pip=12*np.log(pip/440.0)/np.log(2.0)+69
pip


array([ 91.17473907,  95.93542675,  97.89779392,  97.8809569 ,
        97.84224792,  97.82266099,  97.87546399,  97.98102064,
        98.05638998,  96.45707155,  92.70281861,  92.35024081,
        97.89768896, 102.78178298, 104.34484249, 104.3255197 ,
       104.32392098, 102.77728643,  99.17663462,  97.10281945,
        97.17753385,  97.1701985 ,  97.10823059,  97.0611133 ,
        95.60925333,  91.99701135,  89.26428782,  88.832983  ,
        88.84267277,  88.85370169,  91.08755869,  97.66113739,
       103.19417404, 103.14241441,  98.91973346,  96.21125081,
        99.07997048, 103.84120483, 105.54243752, 103.25231616,
       101.1535209 , 103.47939116, 105.33295065, 105.25448357,
       105.16441084, 105.05275681, 104.98146489, 104.73921616,
       104.29034903, 104.07870338, 104.08875142, 104.08418563,
       104.08765691, 104.3801445 , 104.93268153, 105.24265847,
       105.32064392, 105.35713152, 105.31950317, 105.28654189,
       105.72781899, 106.14991107, 105.75207034, 105.35

In [18]:
import scipy.io.wavfile as wav
import numpy as np
import os
import csv
from scipy.fftpack import fft
import pandas
import os
NFFT = 512
F_low = 5
F_high = 8

'''
功能：提取颤音特征
输入：time-pitch序列（间隔5ms）
输出：颤音特征：Likelihood,Rate,Extent
'''
def VibratoFeature(time_pitch):
    NFFT = 512
    pitch_hop = 0.005
    freq = np.arange(0, 1 / (2 * pitch_hop), (1 / (pitch_hop * NFFT)))

    time = time_pitch[:,0]
    pitch = time_pitch[:,1]
    vib_features = np.array([])

    cur_time = 0
    time_length = 0.5 #分析长度为0.5s
    time_hop = 0.25 #滑动为0.25秒
    while cur_time<time[-1]:
        time_stamp = np.array([cur_time,cur_time+time_length])
        
        features = []
        n1 = np.argmin(np.abs(np.subtract(time, time_stamp[0])))
        n2 = np.argmin(np.abs(np.subtract(time, time_stamp[1])))
        pitch_snippet = pitch[n1:n2+1]
        time_snippet = time[n1:n2+1]

        pitch_snippet = pitch_snippet - np.mean(pitch_snippet)

        ### Likelihood
        X = np.abs(fft(pitch_snippet * np.hamming(len(pitch_snippet)), NFFT))
        Xhalf_norm = X[0:int(NFFT / 2)] / sum(X[0:int(NFFT / 2)])
        vib_region = Xhalf_norm[(freq >= F_low) & (freq <= F_high)]
        vib_power = sum(vib_region)
        vib_sum_power = sum(Xhalf_norm)
        vib_power_ratio = vib_power/vib_sum_power
        features.append(vib_power_ratio)

        ### Rate
        zero_crossings = np.where(np.diff(np.sign(pitch_snippet)))[0]
        time_zerocrossings = time_snippet[zero_crossings[0::2]]
        if len(time_zerocrossings) <=1: #不够一个周期==time_zerocrossings不到两个
            if vib_features.size == 0:
                vib_features = np.array([0.,0.,0.])
            else:
                vib_features = np.vstack((vib_features, np.array([0.,0.,0.])))
            cur_time+=time_hop
            continue
        rate = 1.0/((1.0/(len(time_zerocrossings)-1))*sum(np.abs(time_zerocrossings[0:-1]-time_zerocrossings[1:])))
        features.append(rate)

        ### Extent
        pitch_max = []
        for index in range(len(zero_crossings)-1):
            pitch_max.append(max(np.abs(pitch_snippet[zero_crossings[index]:zero_crossings[index+1]])))
        pitch_max = np.array(pitch_max)
        extent =  (0.5/(len(pitch_max)-1))*sum(np.abs(pitch_max[0:-1]+pitch_max[1:]))
        features.append(extent)

        if vib_features.size == 0:
            vib_features = np.array(features)
        else:
            vib_features = np.vstack((vib_features,np.array(features)))
        # print(features)
        cur_time+=time_hop
    return vib_features

VibratoFeature(pip)


IndexError: too many indices for array

In [ ]:
##############################按照文件夹遍历##############################
# base_dir = "G:/data/"
audio_name = "27.wav"
audio_path = "temp/"+audio_name
pitch_path = "temp/"+audio_name[:-4]+"_pitch.csv"
pitch_path2 = "temp/"+audio_name[:-4]+"_pitch_5ms.csv"
note_path = "temp/"+audio_name[:-4]+"_note.csv"
#########################################################################

(fs, ori_sig) = wav.read(audio_path)
window = NFFT / (fs * 1.0)
hop = window / 2.0
#############################如果以前没有提取过pitch##################################
if not os.path.exists(pitch_path2):
    time_hop = (1000/44100)*256
    command = "./AudioBaseTestPitch0625 "+audio_path+" "+pitch_path+" "+note_path+" "+pitch_path2
    print(command)
    os.system(command)
    #########################################################################
    #padding补0
    data = pandas.read_csv(pitch_path2).values
    data = data[np.lexsort(data[:,::-1].T)]
    last_time = (len(ori_sig)/fs)*1000

    #旧的pitch path 2可以删掉
    command2 = "rm "+pitch_path2 
    os.system(command2)

    #以5ms为边界，重写pitch path
    time_hop = 5
    f = open(pitch_path2,'w')
    cur_time = 0 #curtime为时间戳
    j = 0 #j为遍历data的序号
    while cur_time < last_time:
        if j>=data.shape[0] or abs(cur_time-data[j][0])>=(time_hop/2): #以time_hop为边界
            f.write(str(round(cur_time/1000.0,3))+" 0"+'\n')
        else:
            f.write(str(round(data[j][0]/1000.0,3))+" "+str(data[j][1])+'\n')
            j+=1
        cur_time+=time_hop
    f.close()
#############################读取pitch_time##################################
f = open(pitch_path2, 'r')
obj = csv.reader(f, delimiter=' ')
cols = []
for row in obj:
    if len(row)<2: break
    if np.size(cols) == 0:
        cols = [float(elem) for elem in row]
    else:
        cols = np.vstack((cols, [float(elem) for elem in row]))
f.close()
time_pitch = cols
############################提取颤音特征#################################
vib_features = VibratoFeature(time_pitch)

for i in vib_features:
    print(i)